# Day 3: Returns, Volatility & Risk Metrics

## Week 1 - Python for Quantitative Finance

### 🎯 Learning Objectives
- Master all types of return calculations
- Understand volatility estimation methods
- Implement industry-standard risk metrics (VaR, ES, Sharpe, Sortino)
- Calculate and analyze drawdowns

### ⏱️ Time Allocation
- Theory review: 30 min
- Guided exercises: 90 min
- Practice problems: 60 min
- Interview prep: 30 min

---

**Key Interview Topics**: Sharpe ratio, VaR confidence levels, realized vs implied volatility, maximum drawdown

**Author**: ML Quant Finance Mastery  
**Difficulty**: Foundation

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load data
DATA_DIR = Path("../datasets/raw_data")
prices = pd.read_csv(DATA_DIR / "combined_adjusted_close.csv", index_col=0, parse_dates=True)
tickers = ['AAPL', 'MSFT', 'GOOGL', 'SPY', 'JPM']
df = prices[tickers].dropna()

print(f"✅ Data loaded: {df.shape[0]} days, {len(tickers)} stocks")
print(f"📅 Range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")

✅ Data loaded: 1771 days, 5 stocks
📅 Range: 2019-01-02 to 2026-01-16


## 1. Types of Returns

| Type | Formula | When to Use |
|------|---------|-------------|
| Simple | $(P_t - P_{t-1}) / P_{t-1}$ | Portfolio aggregation |
| Log | $\ln(P_t / P_{t-1})$ | Time series analysis |
| Excess | $R_t - R_f$ | Risk-adjusted metrics |
| Relative | $R_t - R_{benchmark}$ | Alpha calculation |

In [8]:
TRADING_DAYS = 252
RISK_FREE_RATE = 0.05 / TRADING_DAYS  # Daily risk-free rate

# Calculate all return types
simple_returns = df.pct_change().dropna()
log_returns = np.log(df / df.shift(1)).dropna()
excess_returns = simple_returns - RISK_FREE_RATE
relative_returns = simple_returns.sub(simple_returns['SPY'], axis=0)

print("📊 RETURN TYPE COMPARISON (AAPL, last 5 days)")
print("=" * 60)
comparison = pd.DataFrame({
    'Simple': simple_returns['AAPL'].tail(),
    'Log': log_returns['AAPL'].tail(),
    'Excess': excess_returns['AAPL'].tail(),
    'vs SPY': relative_returns['AAPL'].tail()
})
print(comparison.round(4))

📊 RETURN TYPE COMPARISON (AAPL, last 5 days)
            Simple     Log  Excess  vs SPY
Date                                      
2026-01-12  0.0034  0.0034  0.0032  0.0018
2026-01-13  0.0031  0.0031  0.0029  0.0051
2026-01-14 -0.0042 -0.0042 -0.0044  0.0007
2026-01-15 -0.0067 -0.0068 -0.0069 -0.0095
2026-01-16 -0.0104 -0.0104 -0.0106 -0.0095


## 2. Volatility Estimation Methods

In [9]:
# Different volatility estimators
aapl_ret = simple_returns['AAPL']

# 1. Historical (realized) volatility
hist_vol = aapl_ret.std() * np.sqrt(TRADING_DAYS)

# 2. Rolling volatility (20-day)
rolling_vol = aapl_ret.rolling(20).std() * np.sqrt(TRADING_DAYS)

# 3. Exponentially weighted volatility
ewm_vol = aapl_ret.ewm(span=20).std() * np.sqrt(TRADING_DAYS)

# 4. Parkinson volatility (if you have High/Low data)
# Uses price range instead of close-to-close
# σ_parkinson = sqrt(1/(4*ln(2)) * ln(H/L)^2)

print("📊 VOLATILITY ESTIMATION")
print("=" * 50)
print(f"Historical (full period): {hist_vol*100:.2f}%")
print(f"Rolling 20-day (latest):  {rolling_vol.iloc[-1]*100:.2f}%")
print(f"EWM 20-day (latest):      {ewm_vol.iloc[-1]*100:.2f}%")
print(f"\n📈 Volatility range over period:")
print(f"   Min:  {rolling_vol.min()*100:.2f}%")
print(f"   Max:  {rolling_vol.max()*100:.2f}%")

📊 VOLATILITY ESTIMATION
Historical (full period): 31.00%
Rolling 20-day (latest):  10.55%
EWM 20-day (latest):      10.90%

📈 Volatility range over period:
   Min:  9.46%
   Max:  107.95%


## 3. Value at Risk (VaR) and Expected Shortfall (ES)

In [4]:
def calculate_var_es(returns: pd.Series, confidence: float = 0.95) -> dict:
    """
    Calculate VaR and Expected Shortfall using multiple methods.
    
    VaR: "With X% confidence, loss won't exceed this amount"
    ES:  "When loss exceeds VaR, the average loss will be this amount"
    """
    alpha = 1 - confidence
    
    # Method 1: Historical (non-parametric)
    var_historical = returns.quantile(alpha)
    es_historical = returns[returns <= var_historical].mean()
    
    # Method 2: Parametric (assumes normal distribution)
    mu = returns.mean()
    sigma = returns.std()
    var_parametric = mu + sigma * stats.norm.ppf(alpha)
    # ES for normal: μ - σ * φ(Φ^(-1)(α)) / α
    es_parametric = mu - sigma * stats.norm.pdf(stats.norm.ppf(alpha)) / alpha
    
    return {
        'var_historical': var_historical,
        'es_historical': es_historical,
        'var_parametric': var_parametric,
        'es_parametric': es_parametric
    }

# Calculate for AAPL
var_es = calculate_var_es(aapl_ret, confidence=0.95)

print("📊 VALUE AT RISK & EXPECTED SHORTFALL (95%)")
print("=" * 60)
print(f"\n{'Method':<20} {'VaR (daily)':<15} {'ES (daily)':<15}")
print("-" * 50)
print(f"{'Historical':<20} {var_es['var_historical']*100:>6.2f}%        {var_es['es_historical']*100:>6.2f}%")
print(f"{'Parametric':<20} {var_es['var_parametric']*100:>6.2f}%        {var_es['es_parametric']*100:>6.2f}%")

print(f"\n💡 Interpretation:")
print(f"   95% VaR of {var_es['var_historical']*100:.2f}% means:")
print(f"   'On 95% of days, loss will be less than {abs(var_es['var_historical'])*100:.2f}%'")

📊 VALUE AT RISK & EXPECTED SHORTFALL (95%)

Method               VaR (daily)     ES (daily)     
--------------------------------------------------
Historical            -2.98%         -4.41%
Parametric            -3.08%         -3.90%

💡 Interpretation:
   95% VaR of -2.98% means:
   'On 95% of days, loss will be less than 2.98%'


## 4. Risk-Adjusted Performance Metrics

In [5]:
def calculate_performance_metrics(returns: pd.Series, rf_rate: float = 0.05) -> dict:
    """Calculate comprehensive performance metrics."""
    
    # Annualize
    ann_return = returns.mean() * TRADING_DAYS
    ann_vol = returns.std() * np.sqrt(TRADING_DAYS)
    
    # Sharpe Ratio: (Return - Rf) / Volatility
    sharpe = (ann_return - rf_rate) / ann_vol
    
    # Sortino Ratio: (Return - Rf) / Downside Volatility
    downside_returns = returns[returns < 0]
    downside_vol = downside_returns.std() * np.sqrt(TRADING_DAYS)
    sortino = (ann_return - rf_rate) / downside_vol
    
    # Calmar Ratio: Return / Max Drawdown
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    calmar = ann_return / abs(max_drawdown)
    
    # Information Ratio (vs SPY)
    benchmark_returns = simple_returns['SPY']
    active_returns = returns - benchmark_returns
    tracking_error = active_returns.std() * np.sqrt(TRADING_DAYS)
    information_ratio = active_returns.mean() * TRADING_DAYS / tracking_error
    
    return {
        'annual_return': ann_return,
        'annual_volatility': ann_vol,
        'sharpe_ratio': sharpe,
        'sortino_ratio': sortino,
        'calmar_ratio': calmar,
        'max_drawdown': max_drawdown,
        'information_ratio': information_ratio
    }

# Calculate for all stocks
print("📊 RISK-ADJUSTED PERFORMANCE METRICS")
print("=" * 80)
print(f"\n{'Ticker':<8} {'Ann Ret':<10} {'Ann Vol':<10} {'Sharpe':<10} {'Sortino':<10} {'Max DD':<10}")
print("-" * 70)

for ticker in tickers:
    metrics = calculate_performance_metrics(simple_returns[ticker])
    print(f"{ticker:<8} {metrics['annual_return']*100:>6.2f}%   {metrics['annual_volatility']*100:>6.2f}%   "
          f"{metrics['sharpe_ratio']:>7.2f}   {metrics['sortino_ratio']:>7.2f}   {metrics['max_drawdown']*100:>6.2f}%")

📊 RISK-ADJUSTED PERFORMANCE METRICS

Ticker   Ann Ret    Ann Vol    Sharpe     Sortino    Max DD    
----------------------------------------------------------------------
AAPL      32.12%    31.00%      0.87      1.21   -33.36%
MSFT      26.53%    28.33%      0.76      1.06   -37.15%
GOOGL     31.12%    31.30%      0.83      1.17   -44.32%
SPY       17.94%    19.73%      0.66      0.80   -33.72%
JPM       23.64%    29.89%      0.62      0.83   -43.63%


## 5. Drawdown Analysis

In [6]:
def analyze_drawdowns(prices: pd.Series, top_n: int = 5) -> pd.DataFrame:
    """
    Analyze drawdown periods and recovery times.
    """
    # Calculate drawdown series
    rolling_max = prices.cummax()
    drawdown = (prices - rolling_max) / rolling_max
    
    # Find drawdown periods
    is_underwater = drawdown < 0
    
    # Find starts and ends of drawdown periods
    starts = is_underwater & ~is_underwater.shift(1).fillna(False)
    ends = ~is_underwater & is_underwater.shift(1).fillna(False)
    
    drawdown_periods = []
    
    start_dates = drawdown.index[starts]
    end_dates = drawdown.index[ends]
    
    for i, start in enumerate(start_dates):
        # Find the trough during this drawdown
        if i < len(end_dates):
            end = end_dates[i] if end_dates[i] > start else drawdown.index[-1]
        else:
            end = drawdown.index[-1]
        
        period_dd = drawdown.loc[start:end]
        trough_date = period_dd.idxmin()
        trough_value = period_dd.min()
        
        # Duration calculations
        days_to_trough = (trough_date - start).days
        days_to_recovery = (end - trough_date).days if end != drawdown.index[-1] else None
        
        drawdown_periods.append({
            'start': start,
            'trough': trough_date,
            'end': end,
            'max_drawdown': trough_value,
            'days_to_trough': days_to_trough,
            'days_to_recovery': days_to_recovery
        })
    
    df_dd = pd.DataFrame(drawdown_periods)
    df_dd = df_dd.nlargest(top_n, 'max_drawdown', keep='first')
    df_dd['max_drawdown'] = df_dd['max_drawdown'].abs()  # Convert to positive for display
    return df_dd.sort_values('max_drawdown', ascending=False)

# Analyze AAPL drawdowns
dd_analysis = analyze_drawdowns(df['AAPL'], top_n=5)

print("📊 TOP 5 DRAWDOWN PERIODS (AAPL)")
print("=" * 80)
print(dd_analysis[['start', 'trough', 'max_drawdown', 'days_to_trough', 'days_to_recovery']].to_string())

📊 TOP 5 DRAWDOWN PERIODS (AAPL)
        start     trough  max_drawdown  days_to_trough  days_to_recovery
65 2023-07-18 2023-07-18      0.001340               0               1.0
21 2019-11-12 2019-11-12      0.000916               0               1.0
74 2024-12-06 2024-12-06      0.000823               0               3.0
27 2019-12-27 2019-12-27      0.000380               0               3.0
13 2019-09-06 2019-09-06      0.000094               0               3.0


## 6. Summary & Key Takeaways

### ✅ What You Learned Today

1. **Return types**: Simple (portfolio), Log (time series), Excess, Relative
2. **Volatility methods**: Historical, Rolling, EWM, Parkinson
3. **Risk metrics**: VaR, ES (CVaR) - historical vs parametric
4. **Performance ratios**: Sharpe, Sortino, Calmar, Information Ratio
5. **Drawdown analysis**: Max DD, recovery times, underwater periods

### 🎯 Interview Tips

- **"Why use Sortino over Sharpe?"** - Sortino only penalizes downside volatility
- **"What's wrong with VaR?"** - Doesn't tell you how bad losses can be (use ES)
- **"Why does Sharpe assume normal returns?"** - It's based on mean/std which are moments of normal dist
- **"What's a good Sharpe ratio?"** - >1 is good, >2 is great, >3 is suspicious

### 📚 Tomorrow: Correlation, Covariance & Factor Analysis